In [ ]:
#numpy serve per le funzioni matematiche
import numpy as np

#matplotlib serve per i grafici
import matplotlib.pyplot as plt

#time serve per misurare il tempo di esecuzione del codice
import time

#exit serve per provocare l'uscita forzata da un codice
from sys import exit

In [ ]:
#creiamo il file contenente i valori delle variabili
f=open('test.txt', 'w') 
f.write('400\n')
f.write('30\n')
f.write('4096\n')
f.write('4096\n')
f.write('-8\n')
f.write('100\n')
f.write('100\n')
f.write('80\n')
f.write('80\n')
f.write('8\n')
f.write('200\n')
f.close



#Leggiamo dal file i valori delle variabili
f = open('test.txt')

var=f.readline()
L=int(var)     #Lunghezza della scatola

var=f.readline()
T=int(var)     #Intervallo temporale

var=f.readline()
N=int(var)     #Numero di punti nello spazio
if N<1000:     #Controlliamo che la discretizzazione spaziale sia abbastanza accurata
    print('il numero di punti nello spazio è troppo piccolo')
    exit()
    
var=f.readline()
M=int(var)    #Numero di punti nel tempo
if M<1000:    #Controlliamo che la discretizzazione temporale sia abbastanza accurata
    print('il numero di punti nello spazio è troppo piccolo')
    exit()
    
var=f.readline()
k0=int(var)    #Velocità di gruppo del pacchetto d'onda

var=f.readline()
E1=int(var)    #Altezza della prima barriera di potenziale

var=f.readline()
E2=int(var)    #Altezza della seconda barriera di potenziale

var=f.readline()
l1=int(var)    #Spessore della prima barriera di potenziale

var=f.readline()
l2=int(var)    #Spessore della seconda barriera di potenziale

var=f.readline()
sigma=int(var) #Varianza

var=f.readline()
freq=int(var)  #Frequenza (mi dice ogni quanto salvare la traiettoria)

f.close


#definiamo altre variabili importanti
dx=L/N         #Discretizzazione spaziale

dt=T/M         #Discretizzazione temporale

x0=L/8         #Valore medio della gaussiana iniziale

x=0            #Estremo spaziale iniziale (lo spazio va da 0 a L)

t=0            #Tempo iniziale

In [ ]:
#Definiamo la forma iniziale del pacchetto d'onda (si tratta di una gaussiana)
iniz=np.zeros( (N), dtype=complex)

psir=np.zeros( (N,M), dtype=complex )

cost= 1/(2*np.pi*sigma*sigma)**(1/4)

for n in range(0,N):
    a=complex(0,k0*x)
    psir[n,0]=cost*((np.e)**(-((x-x0)**2)/(4*sigma*sigma)))*(np.e**(a))
    x=x+dx

In [ ]:
#Definiamo la forma del potenziale
def pot(N,E1,l1,E2,l2):
    V=np.zeros(N)
    for n in range (0,N):
        if n>=N/3 and n<=N/3+l1:
            V[n]=E1
        elif n>=N/2 and n<=N/2+l2:
            V[n]=E2
        else:
            V[n]=0
    return V

#Controlliamo che il potenziale abbia la forma desiderata di 2 barriere di potenziale distinte
if N/3+l1>=N/2:
    print('Le barriere di potenziale si sovrappongono')
    exit()

#Richimiamo il potenziale
V=pot(N,E1,l1,E2,l2)



#Definiamo G1
b=np.zeros(N, dtype=complex)

G1=np.zeros(N, dtype=complex)

for n in range(0,N):
    b[n]=complex(0,-dt/2*V[n])
    G1[n]=(np.e)**(b[n])

In [ ]:
#Definamo G2
k=np.fft.fftfreq(N,2*np.pi/L) #Punti dello spazio dei momenti

a=np.zeros(N, dtype=complex)  

G2=np.zeros(N, dtype=complex)
for n in range(0,N):
    a[n]=complex(0,(dt/2)*k[n]**2)
    G2[n]=(np.e)**(a[n])

In [ ]:
#Facciamo evolvere il sistema
start = time.time()


A=np.zeros(N, dtype=complex)

C=np.zeros(N, dtype=complex)
for m in range (0,M-1):
    for n in range(0,N):
        A[n]=G1[n]*psir[n,m]
    B=np.fft.fft(A) 
    for n in range(0,N):
        C[n]=G2[n]*B[n]
    D=np.fft.ifft(C)
    for n in range(0,N):
        psir[n,m+1]=G1[n]*D[n]
    area = np.trapz((abs(psir[:,m])**2), dx=dx)
    if abs(1-area)>0.002:
        print('la norma non si conserva')
        print('la variazione della norma non è più trascurabile a partire dal tempo', dt*m)
        exit()
    
    
end = time.time()

elapsed = end - start
print(elapsed)

In [ ]:
#Controlliamo la conservazione della norma (che deve essere unitaria)
Err_max=0

i=0

for m in range (0,M-1):
    norm = np.trapz((abs(psir[:,m])**2), dx=dx)
    Err=abs(1-norm)
    if Err>Err_max:
        Err_max=Err
        i=m 
    
print('errore massimo percentile=', Err_max*100,'al tempo',i*dt)

In [ ]:
#Grafichiamo i risultati ottenuti
ascissa=np.arange(0,L,dx)

for m in range (0,M-1):
    if m%1000==0:
        plt.plot(ascissa,(abs(psir[:,m]))**2) 
        plt.plot(ascissa,V)
        plt.ylim(0,0.06)
        plt.xlabel('L')
        plt.ylabel('psi_modulo_quadro')
        plt.title("FUNZIONE D'ONDA")
        plt.show()

In [ ]:
#Salviamo i risultati ottenuti su un file di tipo testo (TASK 1)
file=open('dati.txt','w')

for m in range(0,M-1):
    if m%freq==0:
        file.write('tempo:')
        file.write(str(m*dt) )
        file.write(' traiettoria:')
        file.write(str(abs(psir[:,m])**2))
        file.write('\n')
        
file.close

In [ ]:
#Grafichiamo il pacchetto d'onda a 3 istanti distinti (servirà per verificare l'accurtezza in funzione di dx/dt)

#Grafico al tempo zero
plt.subplot(131) 
plt.title("FUNZIONE D'ONDA")
plt.plot(ascissa,(abs(psir[:,0]))**2) 
plt.plot(ascissa,V)
plt.ylim(0,0.055)
plt.xlabel('L')
plt.ylabel('psi_modulo_quadro')


#Grafico a metà dell'intervallo temporale
plt.subplot(132)   
plt.title("FUNZIONE D'ONDA")
plt.plot(ascissa,(abs(psir[:,int(M/2)]))**2) 
plt.plot(ascissa,V)
plt.ylim(0,0.055)
plt.xlabel('L')
plt.ylabel('psi_modulo_quadro')


#Grafico al tempo finale
plt.subplot(133) 
plt.title("FUNZIONE D'ONDA")
plt.plot(ascissa,(abs(psir[:,M-1]))**2) 
plt.plot(ascissa,V)
plt.ylim(0,0.055)
plt.xlabel('L')
plt.ylabel('psi_modulo_quadro')


plt.subplots_adjust(left=0.01, right=1.6, wspace=0.9, hspace=0.5)

plt.show()                      